<a href="https://colab.research.google.com/github/RasilZiyatdinov/WebAnalyse_PSTU/blob/master/Labs/2024-11-12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import folium
import pandas as pd
from IPython.display import display

file_path = '/content/sample_data/metro.tsv'
metro_data = pd.read_csv(file_path, sep='\t')

metro_data['Название'] = metro_data['Название'].str.replace(r' \(стр.\)', '', regex=True)
metro_data['Название'] = metro_data['Название'].str.replace('ё', 'е')
metro_data['Название'] = metro_data['Название'].str.replace('C', 'С')
metro_data['Название'] = metro_data['Название'].str.replace('c', 'с')
metro_data['Название'] = metro_data['Название'].str.strip()

print(metro_data)

                Название     Широта    Долгота
0               Аэропорт  55.799810  37.534120
1          Академическая  55.688080  37.575010
2           Алексеевская  55.807370  37.638440
3    Александровский сад  55.752190  37.608360
4              Алтуфьево  55.895040  37.586050
..                   ...        ...        ...
160       Воробьевы горы  55.710454  37.558601
161               Выхино  55.715000  37.818020
162              Ясенево  55.605350  37.534940
163         Юго-западная  55.664640  37.484210
164                Южная  55.621220  37.607520

[165 rows x 3 columns]


In [39]:
import requests
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings("ignore", message="Unverified HTTPS request")

url = "https://best-realty.ru/counties/"

response = requests.get(url, verify=False)

if response.status_code == 200:
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, 'html.parser')

    station_district = {}
    districts = soup.find_all("ul", class_="sitemap-ul_2")

    for district in districts:
        district_name_tag = district.find_previous("a", class_="eng_title")
        district_name = district_name_tag.text.strip() if district_name_tag else "Неизвестный округ"
        district_name = district_name.replace('C', 'С').replace('c', 'с')
        stations = district.find_all("a", class_="eng_title")
        for station in stations:
            station_name = station.text.strip().replace('C', 'С').replace('c', 'с')
            station_district[station_name] = district_name

    for station, district in station_district.items():
        print(f"{station}: {district}")
else:
    print("Не удалось получить доступ к странице")



Багратионовская: ЗАО
Киевская: ЗАО
Кропоткинская: ЗАО
Крылатское: ЗАО
Кунцевская: ЗАО
Кутузовская: ЗАО
Молодежная: ЗАО
Парк Победы: ЗАО
Пионерская: ЗАО
Проспект Вернадского: ЗАО
Студенческая: ЗАО
Фили: ЗАО
Юго-Западная: ЗАО
Славянский бульвар: ЗАО
Тропарево: ЗАО
Алексеевская: СВАО
Алтуфьево: СВАО
Бабушкинская: СВАО
Бибирево: СВАО
Ботанический сад: СВАО
ВДНХ: СВАО
Владыкино: СВАО
Медведково: СВАО
Отрадное: СВАО
Савеловская: СВАО
Свиблово: СВАО
Марьина роща: СВАО
Улица Сергея Эйзенштейна: СВАО
Выставочный центр: СВАО
Телецентр: СВАО
Академическая: ЮЗАО
Беляево: ЮЗАО
Новоясеневская: ЮЗАО
Бульвар Адмирала Ушакова: ЮЗАО
Бульвар Дмитрия Донского: ЮЗАО
Бунинская аллея: ЮЗАО
Калужская: ЮЗАО
Каховская: ЮЗАО
Коньково: ЮЗАО
Нахимовский проспект: ЮЗАО
Новые Черемушки: ЮЗАО
Профсоюзная: ЮЗАО
Севастопольская: ЮЗАО
Теплый Стан: ЮЗАО
Улица Горчакова: ЮЗАО
Улица Скобелевская: ЮЗАО
Улица Старокачаловская: ЮЗАО
Университет: ЮЗАО
Ясенево: ЮЗАО
Битцевский парк: ЮЗАО
Измайловская: ВАО
Новогиреево: ВАО
Парти

In [51]:
district_colors = {
    "ЦАО": "red",
    "САО": "blue",
    "ЮЗАО": "green",
    "СВАО": "purple",
    "ЮАО": "orange",
    "ВАО": "pink",
    "ЗАО": "darkblue",
    "СЗАО": "lightgreen",
    "ЮВАО": "beige "
}

moscow_map = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

for index, row in metro_data.iterrows():
    station_name = row["Название"]
    latitude = row["Широта"]
    longitude = row["Долгота"]
    district = station_district.get(station_name.strip(), "Unknown")
    color = district_colors.get(district, "gray")

    folium.Marker(
        location=[latitude, longitude],
        popup=f"{station_name} ({latitude}, {longitude}) - {district}",
        icon=folium.Icon(color=color)
    ).add_to(moscow_map)

map_file_path = '/content/sample_data/moscow_metro_map.html'
moscow_map.save(map_file_path)

display(moscow_map)

<ipython-input-51-4572239e3da5>:25: UserWarning: color argument of Icon should be one of: {'darkgreen', 'darkred', 'black', 'beige', 'pink', 'red', 'green', 'lightred', 'purple', 'lightgreen', 'orange', 'cadetblue', 'darkpurple', 'lightgray', 'white', 'gray', 'blue', 'lightblue', 'darkblue'}.
  icon=folium.Icon(color=color)
